<a href="https://colab.research.google.com/github/alice202122/Thesis/blob/main/DATASEGMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
from os import listdir
from os.path import isfile, join
from skimage import exposure, io, util, measure
import pickle
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import random
import pandas as pd
import seaborn as sns
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

In [8]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [9]:
%cd /content/drive/MyDrive/Tesi

/content/drive/MyDrive/Tesi


In [10]:
!unzip DVCModel1_threshold.zip

Archive:  DVCModel1_threshold.zip
   creating: DVCModel1_threshold/
  inflating: DVCModel1_threshold/1001_guglielmi_anna lucia__318_Angio Retina_OD_2016-06-20_17-55-17_F_1949-01-05_Enface-304x304-Superficial.png  
  inflating: DVCModel1_threshold/1001_guglielmi_anna lucia__318_Angio Retina_OS_2016-06-20_17-58-20_F_1949-01-05_Enface-304x304-Superficial.png  
  inflating: DVCModel1_threshold/1002_Di Carlo_Fonte__427_Angio Retina_OD_2016-10-10_17-19-13_F_1948-07-21_Enface-304x304-Superficial.png  
  inflating: DVCModel1_threshold/1002_Di Carlo_Fonte__427_Angio Retina_OS_2016-10-10_17-22-54_F_1948-07-21_Enface-304x304-Superficial.png  
  inflating: DVCModel1_threshold/1005_mazzarasi_vito__317_Angio Retina_OD_2016-06-20_17-44-18_M_1942-02-20_Enface-304x304-Superficial.png  
  inflating: DVCModel1_threshold/1005_mazzarasi_vito__317_Angio Retina_OS_2016-06-20_17-48-16_M_1942-02-20_Enface-304x304-Superficial.png  
  inflating: DVCModel1_threshold/1006_minoia_vito gianbattista__92_Angio Retina_

In [11]:
!unzip FAZ_Model_threshold.zip

Archive:  FAZ_Model_threshold.zip
   creating: FAZ_Model_threshold/
  inflating: FAZ_Model_threshold/1001_guglielmi_anna lucia__318_Angio Retina_OD_2016-06-20_17-55-17_F_1949-01-05_Enface-304x304-Superficial.png  
  inflating: FAZ_Model_threshold/1001_guglielmi_anna lucia__318_Angio Retina_OS_2016-06-20_17-58-20_F_1949-01-05_Enface-304x304-Superficial.png  
  inflating: FAZ_Model_threshold/1002_Di Carlo_Fonte__427_Angio Retina_OD_2016-10-10_17-19-13_F_1948-07-21_Enface-304x304-Superficial.png  
  inflating: FAZ_Model_threshold/1002_Di Carlo_Fonte__427_Angio Retina_OS_2016-10-10_17-22-54_F_1948-07-21_Enface-304x304-Superficial.png  
  inflating: FAZ_Model_threshold/1005_mazzarasi_vito__317_Angio Retina_OD_2016-06-20_17-44-18_M_1942-02-20_Enface-304x304-Superficial.png  
  inflating: FAZ_Model_threshold/1005_mazzarasi_vito__317_Angio Retina_OS_2016-06-20_17-48-16_M_1942-02-20_Enface-304x304-Superficial.png  
  inflating: FAZ_Model_threshold/1006_minoia_vito gianbattista__92_Angio Retina_

In [12]:
!unzip SVC_Model.zip

Archive:  SVC_Model.zip
   creating: SVC_Model/
  inflating: SVC_Model/1001_guglielmi_anna lucia__318_Angio Retina_OD_2016-06-20_17-55-17_F_1949-01-05_Enface-304x304-Superficial.png  
  inflating: SVC_Model/1001_guglielmi_anna lucia__318_Angio Retina_OS_2016-06-20_17-58-20_F_1949-01-05_Enface-304x304-Superficial.png  
  inflating: SVC_Model/1002_Di Carlo_Fonte__427_Angio Retina_OD_2016-10-10_17-19-13_F_1948-07-21_Enface-304x304-Superficial.png  
  inflating: SVC_Model/1002_Di Carlo_Fonte__427_Angio Retina_OS_2016-10-10_17-22-54_F_1948-07-21_Enface-304x304-Superficial.png  
  inflating: SVC_Model/1005_mazzarasi_vito__317_Angio Retina_OD_2016-06-20_17-44-18_M_1942-02-20_Enface-304x304-Superficial.png  
  inflating: SVC_Model/1005_mazzarasi_vito__317_Angio Retina_OS_2016-06-20_17-48-16_M_1942-02-20_Enface-304x304-Superficial.png  
  inflating: SVC_Model/1006_minoia_vito gianbattista__92_Angio Retina_OD_2016-03-04_09-59-52_M_1937-03-29_Enface-304x304-Superficial.png  
  inflating: SVC_Mode

In [36]:
DATADIR_FAZ = "/content/drive/MyDrive/Tesi/FAZ_Model_threshold"
DATADIR_DEEP = "/content/drive/MyDrive/Tesi/DVCModel1_threshold"
DATADIR_SVP = "/content/drive/MyDrive/Tesi/SVC_Model"


In [52]:
path1 = os.path.join(DATADIR_FAZ)
path2= os.path.join(DATADIR_DEEP)
path3= os.path.join(DATADIR_SVP)
listaFAZ=[]; 
listaSVP=[];
listaDEEP=[];
    
lista3im=[]; 

In [54]:
for img in os.listdir(path1):
   img_path = os.path.join(path1,img)
   im = Image.open(img_path); listaFAZ.append(np.array(im))
lista3im.append(np.array(listaFAZ)); listaFAZ=[]
for img in os.listdir(path2):
   img_path = os.path.join(path2,img)
   im = Image.open(img_path); listaSVP.append(np.array(im))
lista3im.append(np.array(listaSVP)); listaSVP=[]
for img in os.listdir(path3):
   img_path = os.path.join(path3,img)
   im = Image.open(img_path); listaDEEP.append(np.array(im))
lista3im.append(np.array(listaDEEP)); listaDEEP=[]

AttributeError: ignored

In [51]:
lista3im

[array([[[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],
 
        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],
 
        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],
 
        ...,
 
        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,

In [14]:
FullExportedTiffsPath=DATADIR_FAZ+'/'+os.path.join(DATADIR_FAZ).split('.')[0]+'.tif'

In [15]:
FullExportedTiffsPath

'/content/drive/MyDrive/Tesi/FAZ_Model_threshold//content/drive/MyDrive/Tesi/FAZ_Model_threshold.tif'